In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
from copy import deepcopy
from functools import partial
from itertools import chain, combinations, product
from numbers import Number
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser
from scipy.optimize import minimize

from api.risk.var import VaRPnl, get_db_from_clarion_positions, get_fut_trade_spec, get_var_ts, get_db_from_specs, var_ptf, get_clarion_fut_trade_spec, cvar_ptf
from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods, parse_swap_period
from gioutils.tools_clarion import get_clarion_positions
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
    SQL
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_specific_future,
    get_bbg_fut_chain_ticker,
    get_root,
    encode_df_as_image_tag
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)
rng = np.random.default_rng(7)

In [4]:
super_clarion(clarion)
runner = Runner(clarion)

In [5]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-04-05 10:36:02,830 - Authenticating...
2023-04-05 10:36:03,520 - Successfully authenticated


In [6]:
def send_live_var_email(
    var_close, 
    var_intraday, 
    var_new, 
    intraday_group, 
    trades_show,
    subject=f'Live VaR Reduction {pd.Timestamp.now(): %d%b%y %H:%M}'
):
    
    to_email = [
        'giovambattista.perciaccante@brevanhoward.com',
        'menashe.banit@brevanhoward.com'
    ]
    
    font_family = 'Helvetica, Arial, Sans-Serif'
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = from_email
    message["To"] = ','.join(to_email)

    file_name = 'tempVaR'
    dfs = [encode_df_as_image_tag(s, table_conversion='m') for s in [intraday_group, trades_show]]
    html_body = f"<h3 style=\"font-family:{font_family};\">Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR with Intraday (below): {'&nbsp;'*13}{var_intraday: .2%}</h3>"
    html_body = html_body + dfs[0] + '<br>' * 3
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"
    html_body = html_body + dfs[1] 
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())

In [7]:
def send_live_var_email(var_close, var_intraday, var_new, intraday_group, trades_show, test=False):
    to_email = to_email_all if not test else to_email_test
    font_family = 'Helvetica, Arial, Sans-Serif'

    message = MIMEMultipart("alternative")
    message["Subject"] = f'LIVE VaR REDUCTION - {pd.Timestamp.now(): %d%b%y %H:%M}'
    message["From"] = from_email
    message["To"] = ','.join(to_email)

    # dfs = [encode_df_as_image_tag(s) for s in [intraday_group, trades_show]]

    dfs = [df_to_html(s.style.format(millify)) for s in [intraday_group, trades_show]]
    html_body = f"<h3 style=\"font-family:{font_family};\">Close VaR:{'&nbsp;' * 40}{var_close: .2%}</h3>"
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR with Intraday (below): {'&nbsp;' * 13}{var_intraday: .2%}</h3>"
    html_body = html_body + dfs[0] + '<br>' * 3
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR after the below trades:{'&nbsp;' * 11}{var_new: .2%}</h3>"
    html_body = html_body + dfs[1]
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())


### METRICS

In [8]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

### VAR

In [9]:
capital = 300e6
flag = 1.5 / 100 * capital

In [10]:
dv = VaRPnl()

In [11]:
dv.get_data()
f"VaR: {- dv.var() / flag:.2%}"

C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()


'VaR: 18.75%'

In [13]:
cob_ptf = dv.hist.sum(axis=1).rename('ptf')
# cob_ptf = dv.hist \
#     .drop([*dv.ref.query('tradeGroup == "OPT" and instrument == "USD CBT T 10Y"')['positionName']] , axis=1, level=1) \
#     .drop('MM:BOE FEB MAY', axis=1) \
#     .drop(
#         [
#             'USD SOFR CME 3M P 94.812 20230310',
#             'USD SOFR CME 3M P 94.938 20230310',
#         ]
#     , axis=1, level=1) \
#     .sum(axis=1).rename('ptf')

In [51]:
cob_ptf = dv.hist.loc[:, 'MM:EUR OPEN'].sum(axis=1).rename('ptf')

In [52]:
f"Cob VaR: {- cob_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}"

'Cob VaR: 4.53%'

### ADD INTRADAY POSITIONS

In [19]:
df = get_clarion_positions(clarion)
clear_output()

In [20]:
intraday = df.query('isIntraday == True')

In [21]:
intraday_group = intraday.groupby('Description')['Notional'].sum().replace(0, np.nan).dropna().to_frame()
rates_mask = intraday.loc[intraday['Category'] == 'rates', 'Description'].drop_duplicates().to_list()
intraday_rates = intraday_group.loc[intraday_group.index.isin(rates_mask)]
intraday_group

Notional
Description                     
GBP SONIA 3M ICE SEP23       200
USD SOFR 3M CME MAR25        100

### LIVE VAR

In [22]:
db = get_db_from_clarion_positions(intraday)
intraday_trades, res = get_var_ts(db)

In [53]:
current_ptf = pd.concat([cob_ptf, intraday_trades], axis=1).sum(axis=1)
current_ptf = cob_ptf

In [54]:
f"Live VaR: {- current_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}"

'Live VaR: 4.53%'

### CUSTOM ADDITIONS

In [ ]:
add_trades_list = [
    ('sfr', 'h24', 500),
    ('er', 'u23', 800),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [ ]:
add_trades, res = get_var_ts(add_trades_db)

In [ ]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

### AA

In [37]:
add_trades_list = [
    ('ff', 'q23', -1300),
    ('ff', 'z23', 1300),
    ('er', 'h24', 1650),
    ('er', 'h25', -1650),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [38]:
add_trades, res = get_var_ts(add_trades_db)

In [39]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

cob    live  custom
daily  -2.9397 -2.7647 -1.7943
weekly -1.4756 -1.5537 -1.6836

### AA

In [40]:
add_trades_list = [
    ('ff', 'm23', -500),
    ('ff', 'x23', 500),
    # ('er', 'm24', 2000),
    # ('er', 'm25', -2000),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [41]:
add_trades, res = get_var_ts(add_trades_db)

In [42]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

cob    live  custom
daily  -2.9397 -2.7647 -2.3527
weekly -1.4756 -1.5537 -1.6905

### AA

In [43]:
add_trades_list = [
    ('er', 'm24', 800),
    ('er', 'z25', -800),

]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [44]:
add_trades, res = get_var_ts(add_trades_db)

In [45]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

cob    live  custom
daily  -2.9397 -2.7647 -2.5740
weekly -1.4756 -1.5537 -1.5897

### AA

In [46]:
add_trades_list = [
    ('sfi', 'm24', -1000),
    ('sfr', 'm24', 1231),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [47]:
add_trades, res = get_var_ts(add_trades_db)

In [48]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

cob    live  custom
daily  -2.9397 -2.7647 -2.4589
weekly -1.4756 -1.5537 -0.9422

### AA

In [49]:
add_trades_list = [
    ('sfr', 'u23', 1200),
    ('sfr', 'u24', -1200),
    ('er', 'z23', -1100),
    ('er', 'z24', 1100),
    
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [50]:
add_trades, res = get_var_ts(add_trades_db)

In [51]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

cob    live  custom
daily  -2.9397 -2.7647 -2.6947
weekly -1.4756 -1.5537 -1.5797

In [ ]:
display(cob_ptf.nsmallest(1) / capital * 100)
display(current_ptf.nsmallest(1) / capital * 100)
display(pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100)

In [ ]:
display(cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100)
display(current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100)
display(pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100)

In [ ]:
f"Live VaR: {- current_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}"

### NEW TRADES

In [25]:
def bbg_stir_to_clarion(code, date=None):
    date = date or pd.Timestamp.now() 
    year = str(date.year)[-2]
    return f"{code[:-1]}{year}{code[-1]}"

In [66]:
stirs = [
    'er',
    'sfr',
    'sfi',
    'ff'
]

govts = [
    'tu',
    'fv',
    'ty',
    'us',
    'du',
    'oe',
    'rx',
    'ub',
]

In [67]:
gen_futs = [
    *[f"{code}{i}" for code in stirs for i in range(2, 16+1)], 
    *[f"{code}{i}" for code in govts for i in range(1, 1+1)]
]

spec_futs = get_specific_future(gen_futs, bq=bq)

spec_fut_clarion = [fut.split(' ')[0] for fut in spec_futs]
spec_fut_clarion = [bbg_stir_to_clarion(fut) for fut in spec_fut_clarion]

In [68]:
spec_fut_clarion = [fut.split(' ')[0] for fut in spec_futs]
spec_fut_clarion = [bbg_stir_to_clarion(fut) for fut in spec_fut_clarion]

In [69]:
all_fut_list = [(fut[:-3].lower(), fut[-3:].lower()) for fut in spec_fut_clarion]
all_fut_specs = [get_fut_trade_spec(underlying, contract) for underlying, contract in all_fut_list]
all_fut_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in all_fut_list]

In [70]:
all_fut_db = get_db_from_specs(all_fut_specs)

In [71]:
all_fut_ts, res = get_var_ts(all_fut_db)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
from sklearn import set_config
set_config(transform_output = "pandas")

In [ ]:
from gioutils.utils import get_df_reg

In [ ]:
df_reg = get_df_reg(X=all_fut_ts, y=current_ptf, standardize=True)
X, y = df_reg.iloc[:, :-1], df_reg.iloc[:, -1]

In [ ]:
lasso = Lasso(alpha=0.5, fit_intercept=False)
lasso.fit(X=df_reg.iloc[:, :-1], y=df_reg.iloc[:, -1])

In [ ]:
(lasso.coef_ != 0).sum()

In [ ]:
fut_sorted_corr = all_fut_ts.corrwith(current_ptf).abs().sort_values(ascending=False)

In [ ]:
best_trade_list = [(fut[:-3], fut[-3:]) for fut in fut_sorted_corr.index[:4]]

In [ ]:
best_trade_list = [(fut[:-3], fut[-3:]) for fut in X.columns[(lasso.coef_ != 0)]]

In [ ]:
new_trades_list = [
    # ('ff', 'q23'),
    ('ff', 'z23'),
    # ('sfr', 'm23'),
    ('sfr', 'z24'),
    # ('sfr', 'm24'),
    ('er', 'u23'),
    ('er', 'm25'),
    # ('er', 'm23'),
    # ('us', 'm23'),
    # ('oe', 'm23'),
    # ('ty', 'm23'),
    ('us', 'm23')
]

new_trades_list = best_trade_list


new_trades_specs = [get_fut_trade_spec(underlying, contract) for underlying, contract in new_trades_list]
new_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in new_trades_list]

In [ ]:
deltas = runner.run(metrics=[clarion_metrics['delta']], data_list=new_trades_clarion).loc[:, ['id', 'delta']]
clear_output()

In [ ]:
new_trades_db = get_db_from_specs(new_trades_specs)

In [ ]:
new_trades, res = get_var_ts(new_trades_db)

In [ ]:
var_ptf(current_ptf, new_trades, wgt=np.full(shape=new_trades.shape[1], fill_value=1))

### MINIMIZE

In [ ]:
max_size = 10e3

In [ ]:
new_trades = all_fut_ts

In [ ]:
# x0 = rng.normal(size=new_trades.shape[1])
x0 = np.full(shape=new_trades.shape[1], fill_value=-100)
x0 = rng.normal(size=new_trades.shape[1]) * 1000

scales = [*np.hstack([10  ** np.arange(0, np.log10(max_size) + 1), - 10  ** np.arange(0, np.log10(max_size) + 1)])]
max_iter = 10
max_try = max_iter * len(scales)
k= 0 

In [ ]:
res_opt = None
for i in range(max_iter):
    for scale in scales:
        k = k + 1
        x0 = rng.normal(size=new_trades.shape[1]) * scale
        x0[x0 < -max_size] = -max_size
        x0[x0 > max_size] = max_size
        res_opt_ = minimize(
                lambda wgt: cvar_ptf(
                    current_ptf=current_ptf,
                    new_trades=new_trades,
                    wgt=wgt
                ),
                bounds=[(-max_size, max_size) for col in new_trades],
                x0=x0,
                # tol=0.001,
                method='Nelder-Mead', 
                # method='L-BFGS-B',
                # method='SLSQP',
                # constraints=[
                #     {
                #         'type': 'ineq',
                #         'fun': lambda wgt: 0.3 + var(df=d1, wgt=wgt, periods=522, diff=True, scale=-1)
                #     },
                #     {
                #         'type': 'ineq',
                #         'fun': lambda wgt: 1 + var(df=d1, wgt=wgt, periods=522, agg=True, diff=True, scale=-1)
                #     },
                # ],
                # options={
                #     'maxiter': 10000000000000,
                # }
            )
        if res_opt is None:
            res_opt = res_opt_
        
        if res_opt_['fun'] < res_opt['fun']:
            res_opt = res_opt_
        # clear_output()
        print(f"{k} - {max_try} {res_opt['x'].round()} {round(res_opt['fun'], 4)}", end='\r')

### MINIMIZE CVAR

In [39]:
from scipy.optimize import linprog
from gioutils.utils import filter_high_corr
from scipy.special import comb

In [34]:
def min_cvar_linprog(current_ptf, new_trades, q=0.05, min_size=None, max_size=None):
    X = pd.concat([current_ptf, new_trades], axis=1).dropna()
    t, n = X.shape
    c = np.concatenate(
        [   
            np.zeros(shape=n),
            np.full(shape=t, fill_value=(q * t) ** -1),
            np.ones(shape=1)
        ]
    )
    A_ub = np.concatenate(
        [
            -X.to_numpy(),
            -np.eye(t),
            -np.ones(shape=(t, 1)),
        ],
        axis=1
    )
    b_ub = np.zeros(t)
    A_eq = np.zeros(shape=(1, A_ub.shape[1]))
    A_eq[0,0] = 1
    b_eq = 1
    bounds = [
        # None,
        (1.0, 1.0),
        *[(-max_size, max_size) for col in X.columns[1:]],
        *[(0, None) for i in range(t)],
        (None, None)
    ]
    
    r = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    wgt = r['x'][1:X.shape[1]]
    return r, wgt

In [77]:
new_trades = filter_high_corr(all_fut_ts, all_fut_ts.corrwith(current_ptf).abs(), threshold=0.975)

In [74]:
max_size = 10e3
max_iter = int(1e3)
min_ = 1e9
n_trades = 4
total = comb(new_trades.shape[1], n_trades, exact=True)
for i, comb_ in enumerate(combinations(new_trades.columns, min(n_trades, new_trades.shape[1]) if n_trades > 0 else new_trades.shape[1])):
    new_trades_ = new_trades.loc[:, comb_]
    r, wgt = min_cvar_linprog(
        current_ptf=current_ptf, 
        new_trades=new_trades_,
        min_size=-max_size,
        max_size=max_size
    )
    cvar_ = cvar_ptf(current_ptf=current_ptf, new_trades=new_trades_, wgt=wgt)
    var_ = var_ptf(current_ptf=current_ptf, new_trades=new_trades_, wgt=wgt)
    
    if var_ < min_:
        min_ = var_
        wgt_ = wgt
        ress = new_trades_, r, wgt, cvar_, var_
        
    print(f"{i + 1}/{total} - {ress[-3].round()} {round(ress[-2], 4)} {round(ress[-1], 4)}", end='\r')


In [78]:
res_opt = ress[1]
new_trades_ = ress[0]

In [79]:
new_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in [(fut[:-3], fut[-3:]) for fut in new_trades_]]
deltas = runner.run(metrics=[clarion_metrics['delta']], data_list=new_trades_clarion).loc[:, ['id', 'delta']]
clear_output()

In [80]:
var_close = - cob_ptf.quantile(0.05, interpolation='weibull') / flag
var_intraday = - current_ptf.quantile(0.05, interpolation='weibull') / flag

trades = pd.Series(wgt_.round(0), index=new_trades_.columns.str.upper(), name='quantity')
var_new = var_ptf(current_ptf=current_ptf, new_trades=new_trades_, wgt=wgt_)
deltas_opt = pd.Series(deltas['delta'].to_numpy(), index=deltas['id'].str.upper())
delta_usd = (trades * deltas_opt).rename('delta$')

trades_show = pd.concat([trades, delta_usd], axis=1)
sum_ = trades_show.sum().rename('TOTAL')
sum_.iloc[0] = np.nan
sum_= sum_.to_frame().T
trades_show = pd.concat([trades_show, sum_])

In [81]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
# display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
# display(intraday_rates)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades_show)

HTML(value='<h3>Close VaR:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;…

HTML(value='<h3>VaR after the below trades:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; …

quantity  delta$
ERU23      -307   8.39k
ERM25      -191   5.22k
ERH27      -145   3.80k
OEM23       342 -19.84k
TOTAL       NaN  -2.42k

In [ ]:
send_live_var_email(
    var_close, 
    var_intraday, 
    var_new, 
    intraday_rates, 
    trades_show,
    test=True
)

#### Random Search

In [ ]:
wgt_grid = np.arange(-max_size, max_size, 50)

In [ ]:
ress = {}
save_max = 1000
max_iter = 1e5
i = 0
min1 = 1e9
while True:
    i += 1
    wgt = rng.choice(wgt_grid, new_trades.shape[1]) if not isinstance(wgt_grid, dict) else np.asanyarray([rng.choice(wgt_grid[col])  for col in new_trades])
    var_ = var_ptf(current_ptf=current_ptf, new_trades=new_trades, wgt=wgt)
    ress[tuple(wgt)] = var_
    
    if len(ress) > save_max:
        ress.pop(max(ress, key=ress.get))
    
    min2 = min(ress.values())
#     if i == 200:
#         break
    if min2 < min1:
        min1 = min2
        wgt_ = wgt
        
    print(f"{i} - {wgt_} {round(min1, 4)}", end='\r')
    if i + 1 == max_iter:
        break

In [ ]:
res = pd.Series(ress, name='quantity').rename_axis(new_trades.columns).sort_values()

In [ ]:
var_close = - dv.var() / flag

var_intraday = - pd.concat([dv.hist.sum(axis=1).rename('ptf'), ts], axis=1).sum(axis=1).quantile(0.05, interpolation='weibull') / flag
trades = pd.Series(res.reset_index().iloc[0, :-1].to_numpy(), index=res.reset_index().columns[:-1].str.upper(), name='quantity').to_frame()
var_new = res.reset_index().iloc[0, -1]

In [ ]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
# display(db)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades)

In [ ]:
def get_clarion_fut_trade_spec(underlying, contract):
    instrument = 'lfut' if underlying.lower() in lfut_list else 'sfut'
    gen = fut_gen_map.get(underlying.lower())

    return {
        'instrument': instrument,
        'ccy': gen_ccy_map.get(gen),
        'gen': gen,
        'mty_tnr': contract,
        'pay_type': 'pay',
        'notional': 1,
    }

In [ ]:
db_ = get_db_from_clarion_positions(df.query('Category == "rates"'))

In [ ]:
a = bh.bhTsFiRemoteRiskCalc(
    tradesdb=db_,
    startdate=today() - BDay() - parse_offset('2y'),
    enddate=today() - BDay(),
    pblevals=['PV'],
    irbtdbname=SQL.FITradePricer.db,
    irbtdbsrvname=SQL.FITradePricer.server,
    remoteservername=SQL.IRBTServer.server,
    remoteserverport=SQL.IRBTServer.port,
    timeout=120
)

In [ ]:
db_

In [ ]:
a.py.query("Status=='OK' and Metric == 'Rate'")

In [ ]:
new_trades.diff() 

In [ ]:
new_trades / new_trades.iloc[0]